This notebook illustrates the use of different apis for model creation in tensotflow. <br>
APIs used: <br>
1. Sequential api
2. Functional api
3. Sub-classing api

# Iris Dataset

The Iris dataset is a classic dataset in the field of machine learning and statistics, often used as a benchmark for classification algorithms. It consists of 150 iris flower samples from three different species: setosa, versicolor, and virginica.

The task is to classify iris flowers into their correct species based on four measured features:
Sepal length: The length of the flower's sepal in centimeters.
Sepal width: The width of the flower's sepal in centimeters.
Petal length: The length of the flower's petal in centimeters.
Petal width: The width of the flower's petal in centimeters.

Build a model that can accurately predict the species of an iris flower given its four measured features. A good model would have a high accuracy rate, meaning it correctly classifies a large percentage of the iris flowers in the dataset.

### Imports

In [1]:
import os
import shutil
import wandb
import numpy as np
import pandas as pd
import tensorflow as tf
from logzero import logger
from functools import partial
from sklearn.decomposition import PCA 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Layer, Activation
from tensorflow.keras.models import Sequential
# from wandb.integration.keras import WandbCallback

pd.options.mode.use_inf_as_na = True

C:\Users\prapt\AppData\Local\Temp\ipykernel_15672\405057623.py:15: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


### Data Loading

In [2]:
data_config = {
    "data_path": r"data\iris\iris.data",
    "preprocess": True,
    "validate": True,
    "validation_split":0.2,
    "test_split":0.3
}

In [3]:
class DataPipeline:
    def __init__(self):
        logger.info('Data Pipeline set up')
        self.class_mapping = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}

    def load(self, data_path:str):
        """Loading Iris dataset """
        dataset_df = pd.read_csv(data_path, names=[ "sepal_length", "sepal_width", "petal_length", "petal_width", "species"]).sample(frac=1)
        dataset_df["species"] = dataset_df["species"].map(self.class_mapping)
        dataset_df = pd.concat([dataset_df, pd.get_dummies(dataset_df['species'], prefix='species', drop_first=True, dtype=float)], axis=1)
        dataset_df = dataset_df.drop(columns=["species"], axis=1)
        
        # logger.debug( f"Dataset: {dataset_df.head(2)}" )
        logger.info( f"Loaded {len(dataset_df)} records!" )

        return dataset_df

    def preprocess(self, dataset):
        """Pre-process dataset

        Args:
            dataset (_type_): _description_
        """
        dataset["sepal_length"].apply(lambda x: (x - 5.84)/0.83)
        dataset["sepal_width"].apply(lambda x: (x - 3.05)/0.43) 
        dataset["petal_length"].apply(lambda x: (x - 3.76)/1.76)
        dataset["petal_width"].apply(lambda x: (x - 1.20)/0.76) 

        return dataset

    def validate(self, dataset):
        """Validates dataset

        Args:
            dataset (tf.data.Dataset/np.ndarray): .

        Returns:
            isvalid (bool): Is loaded dataset valid
            error (str): Error message in case dataset is not valid
        """
        if len(dataset.columns) != 6:
            return False, "Dataset should have the following columns - [ sepal_length, sepal_width, petal_length, petal_width, species]. One or more columns missing."
        
        null_mask = dataset.isnull()
        if null_mask.values.any():
            return False, f"Null values found at indices - {zip(np.where(null_mask))}"
        
        return True, ""

    def run(self, config:dict):
        """Runner for the complete pipeline

        Args:
            config (dict): .

        Raises:
            Exception: InValidDatasetException when the data schema is not valid 
        """
        dataset = self.load(config["data_path"])
        
        if config.get("validate", False):
            isValid, error = self.validate(dataset)
            if not isValid:
                raise Exception(error)
            else:
                logger.info("Loaded dataset has a valid schema")
        
        split_index = int((1-config["test_split"])*len(dataset))
        train_dataset = dataset.iloc[:split_index]
        test_dataset = dataset.iloc[split_index:]
        logger.debug( f"Records in: Train split - {len(train_dataset)}, Test split - {len(test_dataset)}" )

        if config.get("preprocess", False):
            train_dataset = self.preprocess(train_dataset)
            test_dataset = self.preprocess(test_dataset)
        
        return train_dataset, test_dataset


In [4]:
data_pipeline = DataPipeline()
train_dataset, test_dataset = data_pipeline.run(data_config)

[I 241022 19:38:49 2685342689:3] Data Pipeline set up
[I 241022 19:38:50 2685342689:14] Loaded 150 records!
[I 241022 19:38:50 2685342689:66] Loaded dataset has a valid schema
[D 241022 19:38:50 2685342689:71] Records in: Train split - 105, Test split - 45


### Training

#### Pipeline

In [15]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, model_tags):
        self.accuracy = None
        self.title = "iris-functional_api"
        wandb.run.tags = model_tags

    def on_train_begin(self, logs=
                       None):
        if os.path.exists(f"models/iris/{self.title}"):
            shutil.rmtree(f"models/iris/{self.title}")

    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs)

        if logs and (not self.accuracy or logs["val_acc"] > self.accuracy):
            self.accuracy = logs["val_acc"]
            # tf.saved_model.save(self.model, f"models/iris/{self.title}")
            self.model.save(f"models/iris/{self.title}.h5")
        
    def on_train_end(self, logs=None):
        artifact = wandb.run.log_artifact(f"models/iris/{self.title}.h5", self.title, type="Models")
        wandb.run.link_artifact( artifact=artifact,  target_path="praptipatil01/model-registry/Iris")            

In [8]:
class TrainingPipeline:
    def __init__(self):
        self.sweep_iteration_count = 10
        logger.info('Training Pipeline set up')

    def experiment(self, dataset):
        sweep_id = wandb.sweep(sweep=self.config["sweep"], project="Iris-Species-Classification")
        logger.debug(f"Sweep Id: {sweep_id}")

        wandb.agent(sweep_id, count=self.sweep_iteration_count, function=partial(self.train, dataset))


    def train(self, dataset):    
        if not wandb.run:
            wandb.init(project="Iris-Species-Classification", config=self.config["sweep"])
        
        if not wandb.config and not self.config:
            raise Exception("Configuration not provided for training")

        split_index = int((1-self.config.get("validation_split", 0.2)) * len(dataset))
        train_dataset = dataset.iloc[:split_index]
        val_dataset = dataset.iloc[split_index:]
        logger.debug(f"Number of records: Train -  {len(train_dataset)}, Val - {len(val_dataset)}")
        
        self.model = self.model_builder(num_blocks = wandb.config["num_blocks"], units_1=wandb.config["units_1"], units_2 = wandb.config["units_2"])
        self.model.summary()
        self.model.compile(loss='categorical_crossentropy', optimizer=wandb.config["optimizer"], metrics=[self.config["key_metric"]])
        
        logger.debug(f"Model_name: {self.model.name}")
        if "sequential" in self.model.name or "subclass" in self.model.name:
            train_inputs = train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]]
            validation_inputs = val_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]]
        elif "pca" in self.model.name:
            pca_model = PCA(n_components=2).fit(train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]])
            train_inputs = [train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]], pca_model.transform(train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]]) ]
            validation_inputs = [val_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]], pca_model.transform(val_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]]) ]
        else: 
            raise Exception(f"No input system found for model - {self.model.name}")

        self.model.fit(
            # train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]], train_dataset[["species_1", "species_2"]], 
            train_inputs, train_dataset[["species_1", "species_2"]], 
            batch_size=self.config.get("batch_size", 16), 
            # validation_data = ( val_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]], val_dataset[["species_1", "species_2"]] ),
            validation_data = ( validation_inputs, val_dataset[["species_1", "species_2"]] ),
            epochs=self.config["epochs"],
            callbacks=[CustomCallback( ["subclass_api"] )],
        )
        

    def evaluate(self, X, y, model = None):
        if model:
            self.model  = model

        loss, accuracy = self.model.evaluate( X, y )
        logger.info(f"Loss: {round(loss, 6)*100}")
        logger.info(f"Accuracy: {round(accuracy, 6)*100}")


    def run(self, config:dict, dataset, model_builder=None):
        if config["mode"] not in [ "experiment", "train" ]:
            raise Exception('Invalid mode given. Acceptable modes are [ "experiment", "train" ] ')
        
        if model_builder:
            self.model_builder = model_builder
        self.config = config
        
        if config["mode"] == "experiment":
            self.experiment(dataset)
        elif config["mode"] == "train":
            self.train(dataset)
        
        if not wandb.run:
            wandb.finish()

#### Models

##### 1. Sequential Model
This is a basic sequential model that uses Dense layers to model the problem

In [57]:
def SequentialModel(name, num_blocks=1, *args, **kwargs):
    
    model_layers = []
    for _ in range(num_blocks):
        model_layers += [Dense(32, activation='relu', input_shape=(4,)), Dense(16, activation='relu'), ]
    
    model = Sequential([
        *model_layers,
        Dense(2, activation='softmax') 
    ], name=name)

    return model

In [58]:
train_config = {
    "mode": "experiment",
    "sweep": {
        "name": "wide-net-sequential",
        "description": "This is a widenet sweep to find the write configuration to train the model",
        "method": "bayes",
        "metric": {"goal": "minimize", "name": "validation_loss"},
        "parameters": {
            "learning_rate": {"min": 0.0001, "max": 0.1},
            "optimizer": {"values": ["adam", "sgd"]},
            "num_blocks": { "min": 2, "max": 10, "distribution": "int_uniform" }
        },
    },
    "epochs": 20,
    "batch_size": 150,
    "log_frequency": 1,
    "validation_split": 0.2,
    "key_metric": "acc",
    "baseline": 0.75,
    "metrics": [ "train_acc", "val_acc" ],
}

In [59]:
training_pipeline = TrainingPipeline()
training_pipeline.run(train_config, train_dataset, SequentialModel(name="iris-sequential"))

[I 241020 17:38:57 236141320:4] Training Pipeline set up
c:\Users\prapt\Desktop\Deep Learning\DL_Basics\DL_WS\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Create sweep with ID: m0d9m9ge
Sweep URL: https://wandb.ai/praptipatil01/Iris-Species-Classification/sweeps/m0d9m9ge


[D 241020 17:38:58 236141320:8] Sweep Id: m0d9m9ge
wandb: Agent Starting Run: ornouctq with config:
wandb: 	learning_rate: 0.07079218924411446
wandb: 	num_blocks: 9
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[D 241020 17:39:02 236141320:23] Number of records: Train -  84, Val - 21


Model: "iris-sequential"

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step - acc: 0.4643 - loss: 0.3956 - val_acc: 0.3810 - val_loss: 0.2964
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - acc: 0.5238 - loss: 0.3831INFO:tensorflow:Assets written to: models/iris/iris-sequential\assets


INFO:tensorflow:Assets written to: models/iris/iris-sequential\assets


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - acc: 0.5238 - loss: 0.3831 - val_acc: 0.4762 - val_loss: 0.2876
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - acc: 0.6071 - loss: 0.37 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - acc: 0.6071 - loss: 0.3754 - val_acc: 0.4286 - val_loss: 0.2814
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - acc: 0.6429 - loss: 0.370 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - acc: 0.6429 - loss: 0.3703 - val_acc: 0.4762 - val_loss: 0.2772
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - acc: 0.6905 - loss: 0.36 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - acc: 0.6905 - loss: 0.3674 - val_acc: 0.4762 - val_loss: 0.2742
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - acc: 0.6786 - loss: 0.36 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - acc: 0.6786 - loss: 0.3660 - val_acc: 0.4762 - val_loss: 0.2724
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - acc: 0.6905 - loss: 0.36 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - acc: 0.6905 - loss: 0.3654 - val_acc: 0.4762 - val_loss: 0.2714
Epoch 10/20
1/

wandb: Adding directory to artifact (.\models\iris\iris-sequential)... Done. 0.0s


acc,▇▁▂▄▆▇██████████████
loss,█▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▄▅█▇███████████████
val_loss,█▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.67857
loss,0.35611
val_acc,0.47619
val_loss,0.27175


##### 2. Model using functional API
This model takes 2 inputs in the form of input vectors and top 2 pca components to generate a prediction

In [14]:
def CustomModelWithPCA(name:str, num_blocks:int=1, units_1:int = 16, units_2:int = 8, *args, **kwargs):
    inp1 = Input(shape=(4,))
    inp2 = Input(shape=(2,))

    out1 = Dense(16, activation='relu')(inp1) 
    out2 = Dense(16, activation='relu')(inp2)
    out = Concatenate(axis=1)([out1,out2]) 

    for _ in range(num_blocks):
        out = Dense(units_1, activation='relu')(out)
        out = Dense(units_2, activation='relu')(out)
    
    out = Dense(2, activation='softmax')(out)
    model = Model(inputs=[inp1, inp2], outputs=out, name=name)

    return model

In [15]:
train_config = {
    "mode": "experiment",
    "sweep": {
        "name": "wide-net-pca",
        "description": "This is a widenet sweep. This configuration uses top 2 PCA components to get the result.",
        "group": "iris_functional_api",
        "method": "bayes",
        "metric": {"goal": "minimize", "name": "validation_loss"},
        "parameters": {
            "learning_rate": {"min": 0.0001, "max": 0.1},
            "optimizer": {"values": ["adam", "sgd"]},
            "num_blocks": { "min": 1, "max": 6, "distribution": "int_uniform" },
            "units_1": { "values": list(range(16, 513, 16)) },
            "units_2": { "values": list(range(8, 257, 16)) },
        },
    },
    "epochs": 20,
    "batch_size": 150,
    "log_frequency": 1,
    "validation_split": 0.2,
    "key_metric": "acc",
    "baseline": 0.75,
    "metrics": [ "train_acc", "val_acc" ],
}

In [17]:
training_pipeline = TrainingPipeline()
training_pipeline.run(train_config, train_dataset, CustomModelWithPCA)

[I 241021 21:07:25 3472603808:4] Training Pipeline set up
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('group' was unexpected)


Create sweep with ID: ekxr9ovf
Sweep URL: https://wandb.ai/praptipatil01/Iris-Species-Classification/sweeps/ekxr9ovf


[D 241021 21:07:26 3472603808:8] Sweep Id: ekxr9ovf
wandb: Agent Starting Run: xxiqi0yf with config:
wandb: 	learning_rate: 0.017357028538003402
wandb: 	num_blocks: 5
wandb: 	optimizer: adam
wandb: 	units_1: 448
wandb: 	units_2: 168
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[D 241021 21:07:29 3472603808:23] Number of records: Train -  84, Val - 21


Model: "iris-top2_pca"

...

INFO:tensorflow:Assets written to: models/iris/iris-functional_api\assets


INFO:tensorflow:Assets written to: models/iris/iris-functional_api\assets


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - acc: 0.6667 - loss: 0.4787 - val_acc: 0.8095 - val_loss: 0.5274
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - acc: 0.9167 - loss: 0.47 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - acc: 0.9167 - loss: 0.4779 - val_acc: 0.4762 - val_loss: 0.5222
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - acc: 0.3333 - loss: 0.47 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - acc: 0.3333 - loss: 0.4761 - val_acc: 0.4762 - val_loss: 0.5159
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - acc: 0.3333 - loss: 0.47 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - acc: 0.3333 - loss: 0.4730 - val_acc: 0.4762 - val_loss: 0.5095
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - acc: 0.3333 - loss: 0.469 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - acc: 0.3333 - loss: 0.4693 - val_acc: 0.4762 - val_loss: 0.4972
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - acc: 0.3333 - loss: 0.466 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - acc: 0.3333 - loss: 0.4663 - val_acc: 0.4762 - val_loss: 0.4916
Epoch 7/2

wandb: Adding directory to artifact (.\models\iris\iris-functional_api)... Done. 0.1s


acc,▅█▁▁▁▁▁▄▇▆▁▅▁▅▁▁▅▅▁▁
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▃▂▆█
val_acc,█▁▁▁▁▁▃▇▇▁▂▁▂▁▁▂▂▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▄▂▆█▂
acc,0.33333
loss,31.25541
val_acc,0.47619
val_loss,2.71681


##### 3. Model using Subclassed Models

In [7]:
class MulBiasDense(Layer):
    def __init__(self, units:int = 32, activation = None):
        super(MulBiasDense, self).__init__()
        self.units = units
        self.activation = activation
    
    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(shape = (self.units,), initializer="glorot_uniform", trainable=True)
        self.b_mul = self.add_weight(shape = (self.units,), initializer="glorot_uniform", trainable=True)
    
    def call(self, inputs):
        out = (tf.matmul(inputs, self.w) + self.b) * self.b_mul
        return Activation(self.activation)(out)

In [8]:
def SubclassedModel(name="iris_subclassed", num_blocks:int=1, units_1:int = 32, units_2:int = 16, *args, **kwargs):
    inp = Input(shape=(4,))

    out = inp
    for _ in range(num_blocks):
        out = MulBiasDense(units=units_1, activation='relu')(out)
        out = MulBiasDense(units=units_2, activation='relu')(out)
    
    out = Dense(units=2, activation="softmax")(out)
    model = Model(inputs=inp, outputs=out, name=name)
    return model

In [9]:
train_config = {
    "mode": "experiment",
    "sweep": {
        "name": "wide-net-subclass",
        "description": "This is a widenet sweep. This configuration uses a custom dense layer.",
        "group": "iris_subclassed_api",
        "method": "bayes",
        "metric": {"goal": "minimize", "name": "validation_loss"},
        "parameters": {
            "learning_rate": {"min": 0.0001, "max": 0.1},
            "optimizer": {"values": ["adam", "sgd"]},
            "num_blocks": { "min": 1, "max": 6, "distribution": "int_uniform" },
            "units_1": { "values": list(range(16, 513, 16)) },
            "units_2": { "values": list(range(8, 257, 16)) },
        },
    },
    "epochs": 20,
    "batch_size": 150,
    "log_frequency": 1,
    "validation_split": 0.2,
    "key_metric": "acc",
    "baseline": 0.75,
    "metrics": [ "train_acc", "val_acc" ],
}

In [10]:
training_pipeline = TrainingPipeline()
training_pipeline.run(train_config, train_dataset, SubclassedModel)

[I 241022 09:30:51 1443250203:4] Training Pipeline set up
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('group' was unexpected)


Create sweep with ID: 9re2wssu
Sweep URL: https://wandb.ai/praptipatil01/Iris-Species-Classification/sweeps/9re2wssu


[D 241022 09:30:52 1443250203:8] Sweep Id: 9re2wssu
wandb: Agent Starting Run: xim4sdwv with config:
wandb: 	learning_rate: 0.023117338741545257
wandb: 	num_blocks: 1
wandb: 	optimizer: adam
wandb: 	units_1: 416
wandb: 	units_2: 232
wandb: Currently logged in as: praptipatil01. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[D 241022 09:30:56 1443250203:23] Number of records: Train -  84, Val - 21


Model: "iris_subclassed"

...

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - acc: 0.6190 - loss: 0.4544INFO:tensorflow:Assets written to: models/iris/iris-functional_api\assets


INFO:tensorflow:Assets written to: models/iris/iris-functional_api\assets


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - acc: 0.6190 - loss: 0.4544 - val_acc: 0.8095 - val_loss: 0.4280
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - acc: 0.6190 - loss: 0.45 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - acc: 0.6190 - loss: 0.4543 - val_acc: 0.8095 - val_loss: 0.4282
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - acc: 0.6190 - loss: 0.45 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - acc: 0.6190 - loss: 0.4543 - val_acc: 0.8095 - val_loss: 0.4283
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - acc: 0.6190 - loss: 0.45 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - acc: 0.6190 - loss: 0.4542 - val_acc: 0.8095 - val_loss: 0.4284
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.6190 - loss: 0.45 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - acc: 0.6190 - loss: 0.4541 - val_acc: 0.8095 - val_loss: 0.4286
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - acc: 0.6190 - loss: 0.45 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - acc: 0.6190 - loss: 0.4541 - val_acc: 0.8095 - val_loss: 0.4287
Epoch 7/20
1/1 ━━━

wandb: Adding directory to artifact (.\models\iris\iris-functional_api)... Done. 0.0s


acc,██████████▁▁▁▁▁▁▁▁▁▁
loss,██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▁▁
val_acc,█████████▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██
acc,0.38095
loss,0.45329
val_acc,0.19048
val_loss,0.43048


### Model Comparision
This section compares the best models of the above 3 architectures

In [9]:
pca_model = PCA(n_components=2).fit(train_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]])
training_pipeline = TrainingPipeline()

[I 241022 19:40:54 3612683814:4] Training Pipeline set up


In [16]:
model_registry_path = "praptipatil01/model-registry/Iris"
run = wandb.init()

inputs = {
    "sequential_best": tf.convert_to_tensor(test_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]].values, dtype=tf.float32),
    "functional_api_best": [tf.convert_to_tensor(test_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]].values, dtype=tf.float32), tf.convert_to_tensor(pca_model.transform(test_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]]), dtype=tf.float32)],
    "subclassed_best": tf.convert_to_tensor(test_dataset[["sepal_length",	"sepal_width",	"petal_length",	"petal_width"]].values, dtype=tf.float32),
}
y_true = tf.convert_to_tensor(test_dataset[["species_1", "species_2"]].values, dtype=tf.float32)
accuracy_metric, loss_metric = tf.keras.metrics.CategoricalAccuracy(), tf.keras.losses.CategoricalCrossentropy()

# 
for aliases in [ "sequential_best", "functional_api_best", "subclassed_best" ]:
    artifact = run.use_artifact(f'{model_registry_path}:{aliases}', type='Models')
    artifact_dir = artifact.download()
    model = tf.saved_model.load(artifact_dir).signatures['serving_default']

    if aliases == "functional_api_best":
        y_pred = model(inputs=inputs[aliases][0], inputs_1=inputs[aliases][1])["output_0"]
    else:
        y_pred = model(inputs[aliases])["output_0"]
    accuracy_metric.update_state(y_true, y_pred)
    loss = loss_metric(y_true, y_pred)
    logger.info(f"{aliases.title()}: Loss - {loss}, Accuracy - {accuracy_metric.result().numpy()*100}%")
    accuracy_metric.reset_state()
    logger.debug("")

run.finish()

wandb:   4 of 4 files downloaded.  
[I 241022 19:49:53 630104989:24] Sequential_Best: Loss - 0.3991064429283142, Accuracy - 93.33333373069763%
[D 241022 19:49:53 630104989:26] 
wandb:   4 of 4 files downloaded.  
[I 241022 19:49:55 630104989:24] Functional_Api_Best: Loss - 0.4742332100868225, Accuracy - 71.11111283302307%
[D 241022 19:49:55 630104989:26] 
wandb:   4 of 4 files downloaded.  
[I 241022 19:49:58 630104989:24] Subclassed_Best: Loss - 0.5057439804077148, Accuracy - 68.88889074325562%
[D 241022 19:49:58 630104989:26] 
